In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/IA_PC2/Notebooks

Mounted at /gdrive
/gdrive/My Drive/IA_PC2/Notebooks


# Práctica Calificada 2
## Predicción de tarifas de taxis
El objetivo de esta evaluación es construir un modelo de aprendizaje que sea capaz de predecir la tarifa que cobra un taxi de acuerdo a cierta información de entrada.


In [54]:
import pandas as pd
import numpy as np
print("Pandas = ", pd.__version__)
print("Numpy = ", np.__version__)

Pandas =  1.1.5
Numpy =  1.19.5


# Obteniendo del conjunto de datos

In [13]:
df_train = pd.read_csv("./train.csv", nrows=10000000)

In [14]:
df_train.tail()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
9999995,2012-08-12 01:18:00.000000208,5.7,2012-08-12 01:18:00 UTC,-73.999464,40.728452,-73.993299,40.742100,2
9999996,2013-08-07 10:28:00.000000147,5.5,2013-08-07 10:28:00 UTC,-73.968467,40.759367,-73.964967,40.769027,1
9999997,2013-10-29 08:29:00.00000082,14.0,2013-10-29 08:29:00 UTC,-73.997952,40.733717,-73.973448,40.759122,5
9999998,2012-04-07 16:41:33.0000004,10.5,2012-04-07 16:41:33 UTC,-73.992700,40.752021,-73.964705,40.772849,1
9999999,2010-03-30 19:27:00.00000066,8.5,2010-03-30 19:27:00 UTC,-73.965390,40.768572,-73.998188,40.761073,1


Tenemos las siguientes columnas

* ID: cadena que identifica de manera única a cada registro
* pickup_datetime: timestamp indicando cuando el viaje a empezado
* pickup_longitude: número real indicando la ubicación en longitud en donde el viaje empezó
* pickup_latitude: número real indicando la ubicación en latitud en donde el viaje empezó
* dropoff_longitude: número real indicando la ubicación en longitud en donde el viaje terminó
* dropoff_latitude: número real indicando la ubicación en latitud en donde el viaje terminó
* passenger_count: número entero indicando el número de pasajeros en el servicio de taxi
* fare_amount: número real indicando el costo del taxi. Esta es la variable a predecir.

Borramos la columna ID ya que no es un caracterisitica que nos interese

In [15]:
df_train = df_train.drop(columns=['key'])

In [16]:
df_train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.000000e+07,1.000000e+07,1.000000e+07,9.999931e+06,9.999931e+06,1.000000e+07
mean,1.133854e+01,-7.250775e+01,3.991934e+01,-7.250897e+01,3.991913e+01,1.684793e+00
std,9.799930e+00,1.299421e+01,9.322539e+00,1.287532e+01,9.237280e+00,1.323423e+00
min,-1.077500e+02,-3.439245e+03,-3.492264e+03,-3.426601e+03,-3.488080e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073491e+01,-7.399139e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075263e+01,-7.398016e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396710e+01,4.076712e+01,-7.396367e+01,4.076810e+01,2.000000e+00
max,1.273310e+03,3.457626e+03,3.344459e+03,3.457622e+03,3.351403e+03,2.080000e+02


In [17]:
df_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 7 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   fare_amount        10000000 non-null  float64
 1   pickup_datetime    10000000 non-null  object 
 2   pickup_longitude   10000000 non-null  float64
 3   pickup_latitude    10000000 non-null  float64
 4   dropoff_longitude  9999931 non-null   float64
 5   dropoff_latitude   9999931 non-null   float64
 6   passenger_count    10000000 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 534.1+ MB


De lo anterior se puede observar que tenemos valores nulos, entonces verificamos cuando valores nulos hay por cada columna:

In [18]:
print(df_train.isnull().sum())

fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    69
dropoff_latitude     69
passenger_count       0
dtype: int64


In [19]:
# Eliminando
df_train = df_train.dropna(how='any', axis=0)

# Analizando Fare_amount

In [20]:
df_train[['fare_amount']].describe()

,fare_amount
count,9.999931e+06
mean,1.133849e+01
std,9.799845e+00
min,-1.077500e+02
25%,6.000000e+00
50%,8.500000e+00
75%,1.250000e+01
max,1.273310e+03


Obervamos que el el minimo del monto de tarifa es negativo, veamos cuantos montos de tarifa menores o iguales a cero, tenemos: 

In [21]:
len(df_train[df_train['fare_amount'] <= 0].index)

689

In [22]:
df_train[df_train['fare_amount'] <= 0]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2039,-2.9,2010-03-09 23:37:10 UTC,-73.789450,40.643498,-73.788665,40.641952,1
2486,-2.5,2015-03-22 05:14:27 UTC,-74.000031,40.720631,-73.999809,40.720539,1
10002,0.0,2010-02-15 14:26:01 UTC,-73.987115,40.738808,-74.005911,40.713960,1
13032,-3.0,2013-08-30 08:57:10 UTC,-73.995062,40.740755,-73.995885,40.741357,4
27891,0.0,2015-05-15 21:40:28 UTC,-74.077927,40.805714,-74.077919,40.805721,1
...,...,...,...,...,...,...,...
9891251,-5.7,2010-03-26 22:26:10 UTC,-73.989857,40.739000,-73.995942,40.744332,1
9895476,-2.5,2015-05-10 22:07:39 UTC,-73.789360,40.646481,-73.791451,40.645355,1
9914973,0.0,2010-03-10 15:45:34 UTC,-73.977401,40.763754,-74.185760,40.693433,1
9951612,-2.5,2010-03-24 22:22:10 UTC,-74.010215,40.720048,-74.010188,40.719817,5


Solo queremos los datos donde el monto de la tarifa sea mayor que cero:


In [23]:
df_train = df_train[df_train['fare_amount'] > 0]

In [24]:
df_train[['fare_amount']].describe()

,fare_amount
count,9.999242e+06
mean,1.133966e+01
std,9.798609e+00
min,1.000000e-02
25%,6.000000e+00
50%,8.500000e+00
75%,1.250000e+01
max,1.273310e+03


# Analizando la longitud y latidud

El rango de la latidud es de -90 hasta 90 grados, mientras que el grado de la longitud es de -180 hasta los 180 grados.



In [40]:
df_train = df_train[(df_train['pickup_longitude'] >= -180) & (df_train['pickup_longitude'] <= 180)]

In [46]:
df_train = df_train[(df_train['pickup_latitude'] >= -90) & (df_train['pickup_latitude'] <= 90)]

In [47]:
df_train = df_train[(df_train['dropoff_longitude'] >= -180) & (df_train['dropoff_longitude'] <= 180)]

In [48]:
df_train = df_train[(df_train['dropoff_latitude'] >= -90) & (df_train['dropoff_latitude'] <= 90)]

In [63]:
def distancia(df):
  # Radio medio de la tierra en Km
  R = 6371.0
  # Conversion a radianes
  lt1 = np.radians(df.pickup_latitude)
  lg1 = np.radians(df.pickup_longitude)
  lt2 = np.radians(df.dropoff_latitude)
  lg2 = np.radians(df.dropoff_longitude)
  # Defirencia entre latitudes y longitudes
  dlt = lt2 - lt1
  dlg = lg2 - lg1
  # Haversine
  hav = np.sin(dlt / 2)**2 + np.cos(lt1) * np.cos(lt2) * np.sin(dlg / 2)**2
  d = 2 * R * np.arccos(np.sqrt(hav))
  return d

In [64]:
df_train['distancia'] = distancia(df_train)

In [66]:
df_train = df_train.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

# Analizando pickup_datetime